# Making counter networks

From the data that was collected in the file "Get_Data_From_Riot.ipynb", this code will make several counter networks that represents which champions beat what on different groups of days.

The days are grouped in the following way:
Say we have days 1,2,3,4 for patch 6.23. Networks will be constructed for day groups {1} {1,2} {1,2,3} {1,2,3,4}, {2} {3} {4}, and finally {2,4}. 
This is because we only care about groups of days that occur in order. But this enumeration allows us to have networks for each individual day, the entire set of days, and subsets of concurrent days inbetween.

In [ ]:
################## Import packages n shit ##################
import pandas as pd
import datreant.core as dtr
from datetime import datetime
import numpy as np

################## CHANGE THE PATCH NAME!!! #########################
# Set up path and tree names
net = dtr.Tree('Data/Networks/Challenger_Network_Patch6_24')
net.make()
# Find the matches in the patch I want
matches = dtr.discover('Data/Matches/Challenger_Patch6_24/')
# Find which days are in there
days = matches.categories['Day']
# Find unique days and keep order
indexes = np.unique(days, return_index=True)[1]
sorteddays = [days[index] for index in sorted(indexes)]
# Make an arbitrary list for a loop (probably not needed, I just suck at Python)
indices = list(range(0, len(sorteddays)))




################## Three different loops to make the networks I want #########################
################## Loop 1: Over each single day #########################

for day in sorteddays:

    # Make the patch name for each treant, a folder for days 1-10 or something
    ################## CHANGE THE PATCH NAME!!! #########################
    path0 = 'Data/Networks/Challenger_Network_Patch6_24/days' + str(day) + '-' + str(day)
    thesedays = [day]
    m0 = dtr.Treant(path0)
    m0.path

    ################## CHANGE THE PATCH NAME!!! #########################
    # Write the categories, the patch number, which days, and number of days
    m0.categories['Patch'] = '6_24'
    m0.categories['Number of Days'] = len(thesedays)
    m0.categories['Dates'] = str(thesedays)

    # Then get the matches of each corresponding day group
    groups = matches.categories.groupby("Day")
    data = dtr.Bundle([dtr.Bundle([dtr.Bundle([groups[i] for i in groups if (i in thesedays)]).categories.groupby('Map')['Map.summoners_rift']]).categories.groupby('Queue')['Queue.ranked_solo_queue']])
    norm = len(data)

    ################## Makes the edges for the graph out of existing champions in that data set ##################
    def lots_of_edges(match):
        df = pd.read_csv(match['match.csv'].abspath)
        df = df[['Champion','Win']]
        winners = df[df['Win'] == True]
        winners_names = list(winners['Champion'].unique())
        losers = df[df['Win'] == False]
        losers_names = list(losers['Champion'].unique())
        mixwinnerslosers = pd.DataFrame(columns=['Champion','Win'])
        for winner in winners_names:
            df1 = pd.DataFrame({df.keys()[0]: [winner]*5, df.keys()[1]: [losers_names[0], losers_names[1], losers_names[2], losers_names[3], losers_names[4]]})
            mixwinnerslosers = pd.concat([mixwinnerslosers,df1])
        return mixwinnerslosers

    edges = pd.DataFrame(columns=['Champion','Win'])
    for match in data:
        df2 = lots_of_edges(match)
        edges = pd.concat([edges,df2])

    # Fix column names
    tally = edges.groupby(edges.columns.tolist()).size().reset_index().rename(columns={0:'Weight'})
    tally.rename(columns={'Champion':'Champion A'}, inplace=True)
    tally.rename(columns={'Win':'Champion B'}, inplace=True)

    ################## For each edge, determine the weight as outlined in the lines below this cell ##################
    network = pd.DataFrame(columns=['Champion A','Champion B', 'Weight'])
    championsA = list(tally['Champion A'].unique())
    championsB = list(tally['Champion B'].unique())
    for champb in championsB:
        for champa in championsA:
            AB = tally.loc[tally['Champion B'] == champb].loc[tally['Champion A'] == champa]
            BA = tally.loc[tally['Champion A'] == champb].loc[tally['Champion B'] == champa]
            if len(AB) == 1 and len(BA) == 1:
                wA = AB['Weight'].iloc[0]
                wB = BA['Weight'].iloc[0]
                AB['Weight'] = [(wA/(wA+wB))*((wA+wB)/(norm))]
                BA['Weight'] = [(wB/(wA+wB))*((wA+wB)/(norm))]
            elif len(AB) == 1 and len(BA) == 0: 
                AB['Weight'] = [1*((wA+wB)/(norm))]
            elif len(AB) == 0 and len(BA) == 1: 
                BA['Weight'] = [1*((wA+wB)/(norm))]
            network = pd.concat([network,AB])
            network = pd.concat([network,BA])

    ################## Now save this to a .csv file under the treant ##################
    m0.draw()
    m0['network.csv']
    filename = "network.csv"
    network.to_csv(m0[filename].abspath)
    
    
    
    
    
    
    
################## Loop 2: Over last half of the days #########################

# Make the patch name for each treant, a folder for days 1-10 or something
################## CHANGE THE PATCH NAME!!! #########################
thesedays = sorteddays[-int(round(len(sorteddays)/2)):]
path0 = 'Data/Networks/Challenger_Network_Patch6_24/days' + str(thesedays[0]) + '-' + str(thesedays[-1:][0])
m0 = dtr.Treant(path0)
m0.path

################## CHANGE THE PATCH NAME!!! #########################
# Write the categories, the patch number, which days, and number of days
m0.categories['Patch'] = '6_24'
m0.categories['Number of Days'] = len(thesedays)
m0.categories['Dates'] = str(thesedays)

# Then get the matches of each corresponding day group
groups = matches.categories.groupby("Day")
data = dtr.Bundle([dtr.Bundle([dtr.Bundle([groups[i] for i in groups if (i in thesedays)]).categories.groupby('Map')['Map.summoners_rift']]).categories.groupby('Queue')['Queue.ranked_solo_queue']])
norm = len(data)

################## Makes the edges for the graph out of existing champions in that data set ##################
def lots_of_edges(match):
    df = pd.read_csv(match['match.csv'].abspath)
    df = df[['Champion','Win']]
    winners = df[df['Win'] == True]
    winners_names = list(winners['Champion'].unique())
    losers = df[df['Win'] == False]
    losers_names = list(losers['Champion'].unique())
    mixwinnerslosers = pd.DataFrame(columns=['Champion','Win'])
    for winner in winners_names:
        df1 = pd.DataFrame({df.keys()[0]: [winner]*5, df.keys()[1]: [losers_names[0], losers_names[1], losers_names[2], losers_names[3], losers_names[4]]})
        mixwinnerslosers = pd.concat([mixwinnerslosers,df1])
    return mixwinnerslosers

edges = pd.DataFrame(columns=['Champion','Win'])
for match in data:
    df2 = lots_of_edges(match)
    edges = pd.concat([edges,df2])

# Fix column names
tally = edges.groupby(edges.columns.tolist()).size().reset_index().rename(columns={0:'Weight'})
tally.rename(columns={'Champion':'Champion A'}, inplace=True)
tally.rename(columns={'Win':'Champion B'}, inplace=True)

################## For each edge, determine the weight as outlined in the lines below this cell ##################
network = pd.DataFrame(columns=['Champion A','Champion B', 'Weight'])
championsA = list(tally['Champion A'].unique())
championsB = list(tally['Champion B'].unique())
for champb in championsB:
    for champa in championsA:
        AB = tally.loc[tally['Champion B'] == champb].loc[tally['Champion A'] == champa]
        BA = tally.loc[tally['Champion A'] == champb].loc[tally['Champion B'] == champa]
        if len(AB) == 1 and len(BA) == 1:
            wA = AB['Weight'].iloc[0]
            wB = BA['Weight'].iloc[0]
            AB['Weight'] = [(wA/(wA+wB))*((wA+wB)/(norm))]
            BA['Weight'] = [(wB/(wA+wB))*((wA+wB)/(norm))]
        elif len(AB) == 1 and len(BA) == 0: 
            AB['Weight'] = [1*((wA+wB)/(norm))]
        elif len(AB) == 0 and len(BA) == 1: 
            BA['Weight'] = [1*((wA+wB)/(norm))]
        network = pd.concat([network,AB])
        network = pd.concat([network,BA])

################## Now save this to a .csv file under the treant ##################
m0.draw()
m0['network.csv']
filename = "network.csv"
network.to_csv(m0[filename].abspath)




################## Loop 3: Over day 1-x #########################

indices = list(range(1, len(sorteddays)))
for index in indices:

    # Make the patch name for each treant, a folder for days 1-10 or something
    ################## CHANGE THE PATCH NAME!!! #########################
    thesedays = sorteddays[0:index+1]
    path0 = 'Data/Networks/Challenger_Network_Patch6_24/days' + str(thesedays[0]) + '-' + str(thesedays[-1:][0])
    m0 = dtr.Treant(path0)
    m0.path

    ################## CHANGE THE PATCH NAME!!! #########################
    # Write the categories, the patch number, which days, and number of days
    m0.categories['Patch'] = '6_24'
    m0.categories['Number of Days'] = len(thesedays)
    m0.categories['Dates'] = str(thesedays)

    # Then get the matches of each corresponding day group
    groups = matches.categories.groupby("Day")
    data = dtr.Bundle([dtr.Bundle([dtr.Bundle([groups[i] for i in groups if (i in thesedays)]).categories.groupby('Map')['Map.summoners_rift']]).categories.groupby('Queue')['Queue.ranked_solo_queue']])
    norm = len(data)

    ################## Makes the edges for the graph out of existing champions in that data set ##################
    def lots_of_edges(match):
        df = pd.read_csv(match['match.csv'].abspath)
        df = df[['Champion','Win']]
        winners = df[df['Win'] == True]
        winners_names = list(winners['Champion'].unique())
        losers = df[df['Win'] == False]
        losers_names = list(losers['Champion'].unique())
        mixwinnerslosers = pd.DataFrame(columns=['Champion','Win'])
        for winner in winners_names:
            df1 = pd.DataFrame({df.keys()[0]: [winner]*5, df.keys()[1]: [losers_names[0], losers_names[1], losers_names[2], losers_names[3], losers_names[4]]})
            mixwinnerslosers = pd.concat([mixwinnerslosers,df1])
        return mixwinnerslosers

    edges = pd.DataFrame(columns=['Champion','Win'])
    for match in data:
        df2 = lots_of_edges(match)
        edges = pd.concat([edges,df2])

    # Fix column names
    tally = edges.groupby(edges.columns.tolist()).size().reset_index().rename(columns={0:'Weight'})
    tally.rename(columns={'Champion':'Champion A'}, inplace=True)
    tally.rename(columns={'Win':'Champion B'}, inplace=True)

    ################## For each edge, determine the weight as outlined in the lines below this cell ##################
    network = pd.DataFrame(columns=['Champion A','Champion B', 'Weight'])
    championsA = list(tally['Champion A'].unique())
    championsB = list(tally['Champion B'].unique())
    for champb in championsB:
        for champa in championsA:
            AB = tally.loc[tally['Champion B'] == champb].loc[tally['Champion A'] == champa]
            BA = tally.loc[tally['Champion A'] == champb].loc[tally['Champion B'] == champa]
            if len(AB) == 1 and len(BA) == 1:
                wA = AB['Weight'].iloc[0]
                wB = BA['Weight'].iloc[0]
                AB['Weight'] = [(wA/(wA+wB))*((wA+wB)/(norm))]
                BA['Weight'] = [(wB/(wA+wB))*((wA+wB)/(norm))]
            elif len(AB) == 1 and len(BA) == 0: 
                AB['Weight'] = [1*((wA+wB)/(norm))]
            elif len(AB) == 0 and len(BA) == 1: 
                BA['Weight'] = [1*((wA+wB)/(norm))]
            network = pd.concat([network,AB])
            network = pd.concat([network,BA])

    ################## Now save this to a .csv file under the treant ##################
    m0.draw()
    m0['network.csv']
    filename = "network.csv"
    network.to_csv(m0[filename].abspath)


If A-->B and B-->A exists, 

    Change weights of A-->B and B-->A to WAB/WAB+WBA and WBA/WAB+WBA
    otherwise
    
        If A-->B exists while B-->A doesn't,
            Change weight to 1
            otherwise
            
                If B-->A exists while A-->B doesn't,
                    Change weight to 1

    Let's normalize this by multiplying each edge weight by (#AB + #BA)/#Tot
    So if A beat B in every single game, then A-->B = WAB/WAB+WBA * (#AB + #BA)/#Tot 
                                                    = 100/100 * 100/100 = 1
                                                    as expected.
    B-->A is 0 as expected
    If A-->B is 1 out of 100 games, then A-->B = WAB/WAB+WBA * (#AB + #BA)/#Tot 
                                                    = 1/1 * 1/100 = 1/100
                                                    as expected.
    If A-->B is 50 and B-->A is 50 out of 100 games, then A-->B = WAB/WAB+WBA * (#AB + #BA)/#Tot 
                                                    = 50/100 * 100/100 = 1/2
                                                    as expected.